In [ ]:
import dot_connect

Connect to AWS.

In [ ]:
import boto3

s3 = boto3.client("s3")
s3.list_buckets()

Query Snowflake.

In [ ]:
con = dot_connect.snowflake.connect(database="", schema="magic")
con.cursor().execute("SELECT 1;").fetchall()

Query Snowpark.

In [ ]:
con = dot_connect.snowpark.connect()
con.sql("SELECT 1").show()

Query MySQL.

In [ ]:
con = dot_connect.mysql.connect()

cursor = con.cursor()
cursor.execute("SELECT 1")
cursor.fetchall()

Query Postgres.

In [ ]:
con = dot_connect.postgres.connect()

cursor = con.cursor()
cursor.execute("SELECT 1")
cursor.fetchall()

Testing import checks.

In [ ]:
import pkg_resources

In [ ]:
installed_packages = []

for k, v in pkg_resources.working_set.entry_keys.items():
    if isinstance(v, list) and len(v) > 0:
        print({k: v})
        installed_packages.extend(iter(v))

In [ ]:
installed_packages

In [ ]:
import ibis

In [ ]:
ibis.examples.aml

In [ ]:
examples_registry = ibis.read_csv(
    "https://raw.githubusercontent.com/ibis-project/ibis/master/ibis/examples/registry.txt",
    delim=" ",
    header=False,
    columns={"example": "VARCHAR", "hash": "VARCHAR"},
)

examples = examples_registry.select("example")
examples = examples.mutate(example=examples.example.split(".")[0])
[""] + examples.execute().example.to_list()

In [ ]:
list(ibis.examples._METADATA.keys())

In [ ]:
ibis.examples._METADATA.get("Aids2")

In [ ]:
ibis.examples._METADATA.get("Aids2_raw", "")

In [ ]:
example = ibis.examples.Example("Australian AIDS Survival Data", "Aids2.csv.gz")

In [ ]:
example.fetch().to_pandas()

In [1]:
import dot_connect
import pandas as pd

con = dot_connect.snowflake.connect()

In [2]:
df = pd.read_csv(
    "s3://sfquickstarts/Summit 2022 Keynote Demo/campaign_spend/campaign_spend.csv"
)
df.to_csv("campaign_spend.csv", index=False)

In [17]:
con.cursor().execute(
    "CREATE OR REPLACE FILE FORMAT DOT_CONNECT TYPE = CSV SKIP_HEADER = 1;"
).fetchall()
con.cursor().execute(
    "CREATE OR REPLACE STAGE DOT_CONNECT FILE_FORMAT = DOT_CONNECT;"
).fetchall()

[('Stage area DOT_CONNECT successfully created.',)]

In [18]:
con.cursor().execute(
    "PUT file://campaign_spend.csv @DOT_CONNECT AUTO_COMPRESS=FALSE;"
).fetchall()

[('campaign_spend.csv',
  'campaign_spend.csv',
  13684941,
  13684944,
  'NONE',
  'NONE',
  'UPLOADED',
  '')]

In [26]:
pip install tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
Note: you may need to restart the kernel to use updated packages.


In [29]:
con.cursor().execute(
    """SELECT $1::VARCHAR(60) AS CAMPAIGN, 
              $2::VARCHAR(60) AS CHANNEL, 
              $3::DATE AS DATE, 
              $4::NUMBER(38, 0) AS TOTAL_CLICKS, 
              $5::NUMBER(38, 0) AS TOTAL_COST, 
              $6::NUMBER(38, 0) AS ADS_SERVED
       FROM @DOT_CONNECT;
"""
).fetch_pandas_all().head()

,CAMPAIGN,CHANNEL,DATE,TOTAL_CLICKS,TOTAL_COST,ADS_SERVED
0,winter_sports,video,2012-06-03,213,1762,426
1,sports_across_cultures,video,2012-06-02,87,678,157
2,building_community,search_engine,2012-06-03,66,471,134
3,world_series,social_media,2017-12-28,72,591,149
4,winter_sports,email,2018-02-09,252,1841,473
